In [55]:
import random
import spacy
from spacy.util import minibatch
from spacy.training.example import Example
from spacy.scorer import Scorer
from spacy_layout import spaCyLayout

In [48]:
train_data = [
    ("INV#003281900229170", {"entities": [(0, 19, "RECEIPT NUMBER")]}),
    ("TakeOut Total (incl GST) 26.60", {"entities": [(23, 28, "TOTAL AMOUNT")]}),
    ("Gerbang Alaf Restaurants", {"entities": [(0, 16, "ORG")]}),
    ("ORD#50-REG #19-18/01/2018 10:27:03", {"entities": [(15, 25, "DATE")]}),
    ("Document No TD01167104", {"entities": [(0, 21, "RECEIPT NUMBER")]}),
    ("Date: 25/12/2018 8:13:39 PM", {"entities": [(0, 16, "DATE")]}),
    ("Total: 9.00", {"entities": [(0, 11, "TOTAL AMOUNT")]}),
    ("INDAH GIFT & HOME IECO", {"entities": [(0, 21, "ORG")]}),
    ("19/10/2018 20:49:59 #01", {"entities": [(0, 10, "DATE")]}),
    ("TOTAL AMT. RM 60.31", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
    ("TOTAL RM 33.92", {"entities": [(0, 15, "TOTAL AMOUNT")]}),
    ("Doc No CS00031663 Date 25/12/2018", {"entities": [(0, 17, "RECEIPT NUMBER"), (18, 32, "DATE")]}),
    ("Total Sales (Inclusive of GST) 80.90", {"entities": [(0, 35, "TOTAL AMOUNT")]}),
    ("Date: 25-12-2018 8:13:39 PM", {"entities": [(0, 16, "DATE")]}),
    ("YONGFATT ENTERPRISE", {"entities": [(0, 20, "ORG")]}),
    ("18-11-18 13:58 SH01Z153 T2 R000002902", {"entities": [(0, 8, "DATE")]}),
    ("TOTAL ROUNDED RM 30.90", {"entities": [(0, 21, "TOTAL AMOUNT")]}),
    ("ABC HO TRADING", {"entities": [(0, 14, "ORG")]}),
    ("Total Amount: 31.00", {"entities": [(0, 19, "TOTAL AMOUNT")]}),
    ("Date 09/01/2019 8:01:11 PM", {"entities": [(0, 15, "DATE")]}),
    ("SOON HUAT MACHINERY ENTERPRISE", {"entities": [(0, 30, "ORG")]}),
    ("Doc No. CS00004040 Date 11/01/2019", {"entities": [(0, 18, "RECEIPT NUMBER"), (19, 34, "DATE")]}),
    ("Total Sales 327.00", {"entities": [(0, 18, "TOTAL AMOUNT")]}),
    ("Receipt#: Cs00082552", {"entities": [(0, 20, "RECEIPT NUMBER")]}),
    ("Total (RM): 112.46", {"entities": [(0, 18, "TOTAL AMOUNT")]}),
    ("Salesperson: Date: 12/02/2018", {"entities": [(13, 29, "DATE")]}),
    ("SAM SAM TRADING CO", {"entities": [(0, 18, "ORG")]}),
    ("TOTAL RM 14.10", {"entities": [(0, 13, "TOTAL AMOUNT")]}),
    ("Friday 29-12-2017 Time: 20:17", {"entities": [(6, 16, "DATE")]}),
    ("Good Trust Trading LLC", {"entities": [(0, 22, "ORG")]}),
    ("Grand Total 2,538.968", {"entities": [(0, 21, "TOTAL AMOUNT")]}),
    ("In Words: Omani Rial Two Thousand Five Hundred Thirty Eight and 968/1000 Only", {"entities": [(9, 70, "TOTAL AMOUNT")]}),
    ("Fo Sood Trust Trading LLC", {"entities": [(0, 26, "ORG")]}),
    ("RESTORAN HASSAN BISTRO", {"entities": [(0, 21, "ORG")]}),
    ("Invoice No: 987654321012345", {"entities": [(0, 27, "RECEIPT NUMBER")]}),
    ("Final Amount (incl. Tax) 123.45", {"entities": [(0, 31, "TOTAL AMOUNT")]}),
    ("MegaMart Superstore", {"entities": [(0, 20, "ORG")]}),
    ("ORD#21-REG #05-10/03/2022 14:50:10", {"entities": [(15, 25, "DATE")]}),
    ("Document No: TX981723", {"entities": [(0, 22, "RECEIPT NUMBER")]}),
    ("Date: 08/11/2021 19:30:45", {"entities": [(0, 19, "DATE")]}),
    ("Total: 150.75", {"entities": [(0, 13, "TOTAL AMOUNT")]}),
    ("Date: 09-11-2021 19:30:45", {"entities": [(0, 19, "DATE")]}),
    ("E-Mart Wholesale", {"entities": [(0, 17, "ORG")]}),
    ("DATE OF PURCHASE: 2018-02-01", {"entities": [(19, 29, "DATE")]}),
    ("15/09/2020 12:15:30 #05", {"entities": [(0, 10, "DATE")]}),
    ("TOTAL PAYABLE: 745.00", {"entities": [(0, 22, "TOTAL AMOUNT")]}),
    ("TOTAL DUE: 49.99", {"entities": [(0, 18, "TOTAL AMOUNT")]}),
    ("Doc No: AB123456 Date: 12/06/2019", {"entities": [(0, 18, "RECEIPT NUMBER"), (19, 29, "DATE")]}),
    ("Total (inclusive of Tax): 210.99", {"entities": [(0, 32, "TOTAL AMOUNT")]}),
    ("Global Traders Ltd.", {"entities": [(0, 21, "ORG")]}),
    ("Grand Total: $980.50", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
    ("Total (inclusive of Tax): $210.99", {"entities": [(0, 33, "TOTAL AMOUNT")]}),
    ("21-09-2015 12:15:30 #05", {"entities": [(0, 10, "DATE")]}),
    ("22-08-2021 10:30 AM", {"entities": [(0, 10, "DATE")]}),
    ("TOTAL BALANCE: 1,150.40", {"entities": [(0, 25, "TOTAL AMOUNT")]}),
    ("XYZ Electronics", {"entities": [(0, 15, "ORG")]}),
    ("Grand Total: 980.50", {"entities": [(0, 18, "TOTAL AMOUNT")]}),
    ("Date 05/04/2022 16:45:12", {"entities": [(0, 15, "DATE")]}),
    ("Super Market Express", {"entities": [(0, 22, "ORG")]}),
    ("2022-05-29 10:30 AM", {"entities": [(0, 10, "DATE")]}),
    ("Doc No. XY98765432 Date: 14/07/2020", {"entities": [(0, 18, "RECEIPT NUMBER"), (19, 32, "DATE")]}),
    ("Total Purchase: 450.60", {"entities": [(0, 21, "TOTAL AMOUNT")]}),
    ("Receipt#: ZX321654987", {"entities": [(0, 21, "RECEIPT NUMBER")]}),
    ("DATE OF PURCHASE: 30-10-2019", {"entities": [(18, 28, "DATE")]}),
    ("Total (Amount): 189.75", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
    ("Transaction Date: 03/12/2021", {"entities": [(18, 28, "DATE")]}),
    ("ABC & Co. Stores", {"entities": [(0, 18, "ORG")]}),
    ("TOTAL PAID: 22.90", {"entities": [(0, 17, "TOTAL AMOUNT")]}),
    ("Thursday 17-06-2021 Time: 18:45", {"entities": [(9, 19, "DATE")]}),
    ("Premium Shopping Mall", {"entities": [(0, 23, "ORG")]}),
    ("Total Bill: 3,999.99", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
    ("Amount in Words: Three Thousand Nine Hundred Ninety-Nine and 99/100 Only", {"entities": [(16, 78, "TOTAL AMOUNT")]}),
    ("Bright Retailers Ltd.", {"entities": [(0, 24, "ORG")]}),
    ("Total Bill: $3,999.99", {"entities": [(0, 21, "TOTAL AMOUNT")]}),
    ("RESTAURANT GOURMET DELIGHT", {"entities": [(0, 27, "ORG")]}),
    ("DATE OF PURCHASE: 30/12/2020", {"entities": [(19, 29, "DATE")]}),
]


In [56]:
# nlp = spacy.load('en_core_web_md')
nlp = spacy.blank("en")

In [57]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')


In [58]:
nlp.pipe_names

['ner']

In [59]:
for _, annotations in train_data:
    for ent in annotations['entities']:
        if ent[2] not in ner.labels:
            ner.add_label(ent[2])
            

In [61]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']


with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    
    epochs = 50
    for epoch in range(epochs):
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=8)
        for batch in batches:
            examples = []
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                examples.append(example)
                
            nlp.update(examples, drop=0.2, losses=losses)
        print(f"Epoch {epoch + 1} | Losses : {losses}")
        
nlp.to_disk('invoice_ner_model')

trained_model = spacy.load('invoice_ner_model')

eval_data = train_data

scorer = Scorer()
examples = []
for text, annotations in eval_data:
    doc = trained_model.make_doc(text)
    example = Example.from_dict(doc, annotations)
    example.predicted = trained_model(text)
    examples.append(example)
    
scores = scorer.score(examples)

print("\nModel Evaluation:")
print(f"Precision: {scores['ents_p']:.3f}")
print(f"Recall: {scores['ents_r']:.3f}")
print(f"F1-score: {scores['ents_f']:.3f}")

testing_data = [
    "Receipt #: 9876543210",
    "Total: $45.67",
    "TOTAL AMOUNT USD 150.89",
    "Invoice No. ZX-2024123",
    "Document No: 002345678",
    "Bill Date: 15-07-2022",
    "Invoice Date: 2023/11/05"
]

for text in testing_data:
    doc = trained_model(text)
    print(f"Text : {text}")
    print(f"Entities : {[(ent.text, ent.label_) for ent in doc.ents]}")
    print()

d:\InvoiceFieldExtraction\source\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "ABC & Co. Stores" with entities "[(0, 18, 'ORG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\InvoiceFieldExtraction\source\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Doc No: AB123456 Date: 12/06/2019" with entities "[(0, 18, 'RECEIPT NUMBER'), (19, 29, 'DATE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\InvoiceFieldExtraction\source\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Date: 09-11-2021 19:30:45" wit

Epoch 1 | Losses : {'ner': 194.985400557518}
Epoch 2 | Losses : {'ner': 86.4403624716615}
Epoch 3 | Losses : {'ner': 64.90759425511604}
Epoch 4 | Losses : {'ner': 135.71371671247192}
Epoch 5 | Losses : {'ner': 83.73679831242248}
Epoch 6 | Losses : {'ner': 71.15412385614695}
Epoch 7 | Losses : {'ner': 52.79783821442595}
Epoch 8 | Losses : {'ner': 64.07998326302845}
Epoch 9 | Losses : {'ner': 51.686433875653975}
Epoch 10 | Losses : {'ner': 29.45587922111241}
Epoch 11 | Losses : {'ner': 29.594980403523685}
Epoch 12 | Losses : {'ner': 17.352036210728265}
Epoch 13 | Losses : {'ner': 11.124596911883387}
Epoch 14 | Losses : {'ner': 9.675722893450706}
Epoch 15 | Losses : {'ner': 7.0103980425487915}
Epoch 16 | Losses : {'ner': 2.191668478453265}
Epoch 17 | Losses : {'ner': 0.3060858419521827}
Epoch 18 | Losses : {'ner': 3.7954165245487412}
Epoch 19 | Losses : {'ner': 1.0872230210116078}
Epoch 20 | Losses : {'ner': 0.307058604910259}
Epoch 21 | Losses : {'ner': 0.011576165068057536}
Epoch 22 | L

In [44]:
nlp = spacy.load("en_core_web_md")

doc = nlp("Total : $45.67")

print([token.text for token in doc])

['Total', ':', '$', '45.67']
